In [2]:
import pandas as pd
import numpy as np
import re
import time
import swifter
source_data = pd.read_csv("reddit_raw_data.csv")

In [3]:
print(source_data.shape)
source_data['Timestamp'] = pd.to_datetime(source_data['Timestamp'])
source_data = source_data[(source_data['Timestamp'] > '2017-01-01 00:00:00')]
source_data['Search_Term'] = source_data['Search_Term'].astype(str)
source_data['Content'] =  source_data['Content'].astype(str)
print(source_data.shape)

(115180, 12)
(113026, 12)


In [4]:
def content_cleaning(row):
    final_string = row.Content
    final_string = re.sub(r'\n\s*\n', '\n', final_string, flags=re.MULTILINE)
    final_string = final_string.replace("\n", "")
#     #Weird thing have to do this to convert to unicode then removed by ascii
#     pattern = r"\\u.{4}"
#     mod_string = re.sub(pattern, ' ', final_string)

    #encode() method
    strencode = str(final_string).encode("ascii", "ignore")
    #decode() method
    strdecode = strencode.decode()

    return strdecode
print("Start cleaning at " + time.ctime())
source_data.loc[:,"Content"] = source_data.swifter.apply(content_cleaning, axis=1)

Start cleaning at Tue Apr  4 16:36:19 2023


Pandas Apply:   0%|          | 0/113026 [00:00<?, ?it/s]

In [5]:

print("Start Case Fold at " + time.ctime())
source_data.loc[:,"Search_Term"] = source_data["Search_Term"].str.lower()
source_data.loc[:,"Content"] = source_data["Content"].str.lower()

print("End Case Fold at " + time.ctime())
print(source_data.shape)

Start Case Fold at Tue Apr  4 16:36:22 2023
End Case Fold at Tue Apr  4 16:36:22 2023
(113026, 12)


In [6]:

def find_exact_match_no_split(row):
    search_token = row.Search_Term
    matched = True
    search_token = r'\b' + re.escape(search_token) + r'\b'
    found = re.search(search_token, row.Content)
    if not found:
        matched = False
            
    return matched






In [7]:
print("Finding Matching Word Start at " + time.ctime())

source_data.loc[:,"Matching_Word"] = source_data.swifter.apply(find_exact_match_no_split, axis=1)
print("Finding Matching Word End at " + time.ctime())

Finding Matching Word Start at Tue Apr  4 16:36:25 2023


Pandas Apply:   0%|          | 0/113026 [00:00<?, ?it/s]

Finding Matching Word End at Tue Apr  4 16:36:30 2023


In [8]:
source_data = source_data[source_data['Matching_Word'] == True]
source_data.head()


,User ID,Type,Content ID,URL,Reply to Content ID,Thread Selftext,Subreddit,Title,Content,Timestamp,Score,Search_Term,Matching_Word
176,SG_wormsblink,Thread,11apg1h,/r/singapore/comments/11apg1h/claims_that_midd...,j9tbfx6,NaN,all,Claims that middle-income Singaporeans are ove...,"looking at this article literally, i think leo...",2023-02-24 11:42:53,2023-02-24 12:22:53,middle income,True
188,SG_wormsblink,Thread,11apg1h,/r/singapore/comments/11apg1h/claims_that_midd...,j9uc9rw,NaN,all,Claims that middle-income Singaporeans are ove...,look at it this way.your cpf savings grow at a...,2023-02-24 11:42:53,2023-02-24 16:53:29,middle income,True
208,RecetasEmpanadas,Thread,11e84d9,/r/asklatinamerica/comments/11e84d9/what_do_yo...,jad3386,"Hi! Recently, I've been interested in the econ...",all,What do you think of the middle-income trap in...,i saw people saying that brazil cant escape th...,2023-02-28 14:26:58,2023-02-28 15:57:13,middle income,True
216,RecetasEmpanadas,Thread,11e84d9,/r/asklatinamerica/comments/11e84d9/what_do_yo...,jagvs5z,"Hi! Recently, I've been interested in the econ...",all,What do you think of the middle-income trap in...,how 200 usd monthly average income can be cons...,2023-02-28 14:26:58,2023-03-01 10:04:37,middle income,True
219,RecetasEmpanadas,Thread,11e84d9,/r/asklatinamerica/comments/11e84d9/what_do_yo...,jae1niy,"Hi! Recently, I've been interested in the econ...",all,What do you think of the middle-income trap in...,"- i do belive in the middle income trap, parti...",2023-02-28 14:26:58,2023-02-28 19:36:40,middle income,True


In [9]:
print(source_data.shape)


(7261, 13)


In [10]:
source_data.to_csv("reddit_clean_data.csv")